<a href="https://colab.research.google.com/github/byudhau23/Information-Retrival/blob/main/Praktikum_2_Bimo_Yudha_Utama_0110222057.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [37]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import re
import math
from tqdm import tqdm

In [25]:
import gdown

file_id = "1ysv88LcYDyjYrSrC_Fke4JThiv2BuP9y"
gdown.download(id=file_id, output="data.xml", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ysv88LcYDyjYrSrC_Fke4JThiv2BuP9y
From (redirected): https://drive.google.com/uc?id=1ysv88LcYDyjYrSrC_Fke4JThiv2BuP9y&confirm=t&uuid=9f04efec-9e27-4647-9dc1-1bf4f2a31d68
To: /content/data.xml
100%|██████████| 2.56M/2.56M [00:00<00:00, 199MB/s]


'data.xml'

In [24]:
# Inspect the content of the downloaded file to confirm it's HTML or XML
with open('data.xml', 'r') as f:
    print(f.read(500)) # Print the first 500 characters for inspection

# If confirmed to be XML, uncomment the following lines to read it with pandas
# df = pd.read_xml("data.xml")
# df.head()

<?xml version='1.0' encoding='utf-8'?>
<data_berita><berita id="01"><sumber>kompas.com</sumber><tanggal>2015/07/01</tanggal><kategori>Teknologi</kategori><judul>Ponsel Huawei Honor 4C Dibanderol Rp 2,2 Juta</judul><isi>JAKARTA, KOMPAS.com  Ponsel Android Huawei Honor 4C resmi meluncur di pasar Indonesia, Rabu (1/7/2015). Perusahaan asal Tiongkok itu merancangnya sebagai ponsel kelas menengah dengan banderol harga Rp 2,3 juta.Kami merancangnya untuk pengguna dari usia 18-35 tahun. Ponsel ini suda


In [30]:
df = pd.read_xml("data.xml")
df.drop(columns = ['sumber', 'tanggal', 'link'], inplace = True)
df['judul'] = df['judul'].apply(casaFolding)
df['isi'] = df['isi'].apply(casaFolding)
df['judul'] = df['judul'].apply(punctuationRemoval)
df['isi'] = df['isi'].apply(punctuationRemoval)
df.head()

,id,kategori,judul,isi,jumlahkata
0,1,Teknologi,ponsel huawei honor 4c dibanderol rp 22 juta,jakarta kompascom ponsel android huawei honor...,315
1,2,Teknologi,asosiasi rpp ecommerce tidak sesuai hasil diskusi,jakarta kompascom sejak 2013 wacana tentang r...,419
2,3,Teknologi,pemesan iphone jadi sabun karyawan pesaing lazada,jakarta kompascom danis darusman pelanggan laz...,265
3,4,Teknologi,autofeather failure momok bagi pesawat balingb...,kompascom salah satu momok yang dihadapi dalam...,481
4,5,Teknologi,laptop bezel tertipis di dunia masuk indonesia,jakarta kompascom resmi diperkenalkan pada aj...,227


In [14]:
df['kategori'].unique()

array(['Teknologi', 'Bisnis Ekonomi', 'Nasional', 'Olahraga', 'Travel',
       'Bola', 'Otomotif', 'Lifestyle'], dtype=object)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1502 non-null   int64 
 1   kategori    1502 non-null   object
 2   judul       1502 non-null   object
 3   isi         1502 non-null   object
 4   jumlahkata  1502 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 58.8+ KB


In [16]:
df.isnull().sum()

,0
id,0
kategori,0
judul,0
isi,0
jumlahkata,0


In [18]:
df.duplicated().sum()

np.int64(0)

In [20]:
def casaFolding(text) :
  text = text.lower()
  return text


contoh = "Hibban Sang Pencari Secret"
print(f'original: {contoh}')
print(f'folding: {casaFolding(contoh)}')

original: Hibban Sang Pencari Secret
folding: hibban sang pencari secret


In [22]:
def punctuationRemoval (text) :
  text = re.sub(r'[^\w\s]', '', text)
  return text

contoh = "Hibban Sang Pencari Secret!!!"
print(f'original: {contoh}')
print(f'punctuation removal: {punctuationRemoval(contoh)}')

original: Hibban Sang Pencari Secret!!!
punctuation removal: Hibban Sang Pencari Secret


In [28]:
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def remove_stopwords (text):
  tokens = text.split()
  filtered_tokens = []

  for word in tokens :
    if word not in stop_words :
      filtered_tokens.append(word)

  cleaned_text = " ".join(filtered_tokens)
  return cleaned_text

contoh_teks = "saya sedang belajar bahasa inggris"
print(f'original: {contoh_teks}')
print(f'stopwords removal: {remove_stopwords(contoh_teks)}')


original: saya sedang belajar bahasa inggris
stopwords removal: belajar bahasa inggris


In [40]:
df['clean'] = df['isi'].apply(casaFolding)
df['clean'] = df['clean'].apply(punctuationRemoval)
df.head()

,id,kategori,judul,isi,jumlahkata,clean
0,1,Teknologi,ponsel huawei honor 4c dibanderol rp 22 juta,jakarta kompascom ponsel android huawei honor...,315,jakarta kompascom ponsel android huawei honor...
1,2,Teknologi,asosiasi rpp ecommerce tidak sesuai hasil diskusi,jakarta kompascom sejak 2013 wacana tentang r...,419,jakarta kompascom sejak 2013 wacana tentang r...
2,3,Teknologi,pemesan iphone jadi sabun karyawan pesaing lazada,jakarta kompascom danis darusman pelanggan laz...,265,jakarta kompascom danis darusman pelanggan laz...
3,4,Teknologi,autofeather failure momok bagi pesawat balingb...,kompascom salah satu momok yang dihadapi dalam...,481,kompascom salah satu momok yang dihadapi dalam...
4,5,Teknologi,laptop bezel tertipis di dunia masuk indonesia,jakarta kompascom resmi diperkenalkan pada aj...,227,jakarta kompascom resmi diperkenalkan pada aj...


In [46]:
def compute_word_frequency(text):
    words = word_tokenize(text)
    frequency_table = {}

    for word in words :
        if word in frequency_table :
            frequency_table[word] += 1
        else :
            frequency_table[word] = 1

    return frequency_table

In [52]:
nltk.download('punkt_tab')
df['word_frequency'] = df['clean'].apply(compute_word_frequency)
df.head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,id,kategori,judul,isi,jumlahkata,clean,word_frequency
0,1,Teknologi,ponsel huawei honor 4c dibanderol rp 22 juta,jakarta kompascom ponsel android huawei honor...,315,jakarta kompascom ponsel android huawei honor...,"{'jakarta': 1, 'kompascom': 1, 'ponsel': 5, 'a..."
1,2,Teknologi,asosiasi rpp ecommerce tidak sesuai hasil diskusi,jakarta kompascom sejak 2013 wacana tentang r...,419,jakarta kompascom sejak 2013 wacana tentang r...,"{'jakarta': 2, 'kompascom': 1, 'sejak': 1, '20..."
2,3,Teknologi,pemesan iphone jadi sabun karyawan pesaing lazada,jakarta kompascom danis darusman pelanggan laz...,265,jakarta kompascom danis darusman pelanggan laz...,"{'jakarta': 1, 'kompascom': 1, 'danis': 12, 'd..."
3,4,Teknologi,autofeather failure momok bagi pesawat balingb...,kompascom salah satu momok yang dihadapi dalam...,481,kompascom salah satu momok yang dihadapi dalam...,"{'kompascom': 1, 'salah': 2, 'satu': 3, 'momok..."
4,5,Teknologi,laptop bezel tertipis di dunia masuk indonesia,jakarta kompascom resmi diperkenalkan pada aj...,227,jakarta kompascom resmi diperkenalkan pada aj...,"{'jakarta': 2, 'kompascom': 1, 'resmi': 1, 'di..."


In [56]:
N = len(df)

document_frequency = {}

for word_freq in df ['word_frequency']:
  for word, freq in word_freq.items():
    if word in document_frequency:
      document_frequency[word] += 1
    else :
      document_frequency[word] = 1

idf = {}

for word, df_t in document_frequency.items():
  idf[word] = math.log((1+N)/(1+df_t))

list(idf.items())[:10]

[('jakarta', 0.8663289956061171),
 ('kompascom', 2.3180061159888594),
 ('ponsel', 4.179724173823825),
 ('android', 5.117993812416755),
 ('huawei', 5.523458920524919),
 ('honor', 5.705780477318874),
 ('4c', 5.928924028633084),
 ('resmi', 2.519427844156233),
 ('meluncur', 4.830311739964974),
 ('di', 0.10366165643917329)]

In [59]:
def compute_tf_idf(word_freq, idf_dict):
  tf_idf = {}

  for word, freq in word_freq.items():
    if word in idf_dict:
      tf_idf[word] = freq * idf_dict[word]
  return tf_idf

tf_idf_list = []

for freq_dict in df['word_frequency']:
  tf_idf_list.append(compute_tf_idf(freq_dict, idf))

df['tf_idf'] = tf_idf_list
df.head()

,id,kategori,judul,isi,jumlahkata,clean,word_frequency,tf_idf
0,1,Teknologi,ponsel huawei honor 4c dibanderol rp 22 juta,jakarta kompascom ponsel android huawei honor...,315,jakarta kompascom ponsel android huawei honor...,"{'jakarta': 1, 'kompascom': 1, 'ponsel': 5, 'a...","{'jakarta': 0.8663289956061171, 'kompascom': 2..."
1,2,Teknologi,asosiasi rpp ecommerce tidak sesuai hasil diskusi,jakarta kompascom sejak 2013 wacana tentang r...,419,jakarta kompascom sejak 2013 wacana tentang r...,"{'jakarta': 2, 'kompascom': 1, 'sejak': 1, '20...","{'jakarta': 1.7326579912122342, 'kompascom': 2..."
2,3,Teknologi,pemesan iphone jadi sabun karyawan pesaing lazada,jakarta kompascom danis darusman pelanggan laz...,265,jakarta kompascom danis darusman pelanggan laz...,"{'jakarta': 1, 'kompascom': 1, 'danis': 12, 'd...","{'jakarta': 0.8663289956061171, 'kompascom': 2..."
3,4,Teknologi,autofeather failure momok bagi pesawat balingb...,kompascom salah satu momok yang dihadapi dalam...,481,kompascom salah satu momok yang dihadapi dalam...,"{'kompascom': 1, 'salah': 2, 'satu': 3, 'momok...","{'kompascom': 2.3180061159888594, 'salah': 3.1..."
4,5,Teknologi,laptop bezel tertipis di dunia masuk indonesia,jakarta kompascom resmi diperkenalkan pada aj...,227,jakarta kompascom resmi diperkenalkan pada aj...,"{'jakarta': 2, 'kompascom': 1, 'resmi': 1, 'di...","{'jakarta': 1.7326579912122342, 'kompascom': 2..."


In [62]:
# Pilih dokumen berdasarkan id berita
id_berita = 1   # ubah sesuai id berita yang ingin ditampilkan
row = df[df['id'] == id_berita].iloc[0]

# Ambil data TF dan TF-IDF dari dokumen tersebut
tf_data    = row['word_frequency']
tfidf_data = row['tf_idf']

rows = []
for term, tf in tf_data.items():
    idf_value   = idf.get(term, 0)
    tfidf_value = tfidf_data.get(term, 0)

    rows.append({
        'term':  term,
        'tf':    tf,
        'idf':   round(idf_value, 4),
        'tfidf': round(tfidf_value, 4)
    })

# Buat DataFrame detail TF-IDF
df_tfidf_detail = pd.DataFrame(rows)

# Mengurutkan dari nilai tfidf terbesar
df_tfidf_detail = df_tfidf_detail.sort_values(by='tfidf', ascending=False)

print(f"Hasil TF-IDF untuk id berita = {id_berita}\n")
print(df_tfidf_detail.to_string(index=False))


Hasil TF-IDF untuk id berita = 1

             term  tf    idf   tfidf
               4c   5 5.9289 29.6446
            honor   4 5.7058 22.8231
           ponsel   5 4.1797 20.8986
           darren   3 6.6221 19.8662
           huawei   3 5.5235 16.5704
         prosesor   3 4.7503 14.2508
     merancangnya   2 6.6221 13.2441
            photo   2 6.2166 12.4332
       megapiksel   2 5.9289 11.8578
          refocus   2 5.9289 11.8578
         snapshot   2 5.9289 11.8578
     penjualannya   2 5.9289 11.8578
             fast   2 5.7058 11.4116
            ultra   2 5.7058 11.4116
           selfie   2 5.7058 11.4116
         pengguna   3 3.7889 11.3666
               gb   2 5.5235 11.0469
             mode   2 5.3693 10.7386
         memotret   2 5.2358 10.4716
          android   2 5.1180 10.2360
           kamera   2 4.3708  8.7416
            fitur   2 4.0963  8.1927
             foto   2 4.0571  8.1142
      menggunakan   3 2.3454  7.0362
        konduktor   1 6.6221  6.6221
    